### Import

In [1]:
import pandas as pd

### Preview data

In [2]:
raw_df = pd.read_csv('./data/raw_data.csv')
raw_df.head()

,company,company_size,job_title,level,domain,yoe_total,yoe_at_company,base,stock,bonus,total_compensation,location
0,Logitech,"7,250",Software Engineer,I4,Testing (SDET),10 yrs,5 yrs,190K,10K,NaN,"$200,000",San Francisco Bay Area
1,Logitech,"7,250",Software Engineer,I2,ML / AI,4 yrs,3 yrs,126K,NaN,7K,"$133,000","Vancouver, WA"
2,Logitech,"7,250",Software Engineer,I3,Testing (SDET),11+ yrs,11+ yrs,120K,5K,12K,"$137,000","San Francisco, CA"
3,Logitech,"7,250",Software Engineer,hidden,hidden,0-1 yrs,0-1 yrs,90K,NaN,NaN,"$90,000","Newark, CA"
4,Logitech,"7,250",Software Engineer,I4,Production,8 yrs,8 yrs,100K,10K,NaN,"$110,000","Hsin-chu, TP, Taiwan"


### Data shape

In [3]:
shape = raw_df.shape
print(f'Raw data shape: {shape}')

Raw data shape: (1279, 12)


### Col meaning

 **Give the definition of the col meaning**

In [4]:
col_meaning_df = pd.read_csv('./data/schema.csv')
col_meaning_df

,Query API,Meaning
0,company,Company Name
1,company_size,Company Size
2,job_title,Job Title
3,level,Level (Based on company's internal level system)
4,domain,Domain (Industry)
5,yoe_total,Years of Experience
6,yoe_at_company,Years of Experience at Company
7,base,Base Salary (USD - Annual)
8,stock,Stock (USD - Annual)
9,bonus,Bonus (USD - Annual)


### Observation

#### Types

In [5]:
dtypes = raw_df.dtypes
dtypes

company               object
company_size          object
job_title             object
level                 object
domain                object
yoe_total             object
yoe_at_company        object
base                  object
stock                 object
bonus                 object
total_compensation    object
location              object
dtype: object

#### Convert `object` type to the correct type

In [6]:
# convert the company_size column from object (e.g. 2,000) to int (e.g. 2000)
raw_df['company_size'] = raw_df['company_size'].str.replace(',', '').astype(int)
raw_df['company_size'].head()

0    7250
1    7250
2    7250
3    7250
4    7250
Name: company_size, dtype: int32

In [7]:
# convert the base, stock, bonus columns from object (e.g. 80K) to int (e.g. 80000), skip the NaN values
# K -> 1000, M -> 1000000
raw_df['base'] = raw_df['base'].str.replace('K', '').str.replace('M', '000').astype(float) * 1000
raw_df['stock'] = raw_df['stock'].str.replace('K', '').str.replace('M', '000').astype(float) * 1000
raw_df['bonus'] = raw_df['bonus'].str.replace('K', '').str.replace('M', '000').astype(float) * 1000
raw_df[['base', 'stock', 'bonus']].head()

,base,stock,bonus
0,190000.0,10000.0,NaN
1,126000.0,NaN,7000.0
2,120000.0,5000.0,12000.0
3,90000.0,NaN,NaN
4,100000.0,10000.0,NaN


In [8]:
# convert the total_compensation column from object (e.g. $240,000) to int (e.g. 240000)
raw_df['total_compensation'] = raw_df['total_compensation'].str.replace('$', '').str.replace(',', '').astype(float)
raw_df['total_compensation'].head()

0    200000.0
1    133000.0
2    137000.0
3     90000.0
4    110000.0
Name: total_compensation, dtype: float64

In [9]:
# deal with `yoe_total` and `yoe_at_company` columns
# format 1: `n yrs` -> n
# format 2: `n+ yrs` -> n
# format 3: `m-n yrs` -> drop

def convert_yoe(yoe):
	# format 2
	if '+' in yoe:
		return int(yoe.split('+')[0])
	# format 3
	elif '-' in yoe:
		return None
	# format 1
	elif ' ' in yoe:
		return int(yoe.split(' ')[0])
	else:
		return int(yoe)
	
raw_df['yoe_total'] = raw_df['yoe_total'].apply(convert_yoe)
raw_df['yoe_at_company'] = raw_df['yoe_at_company'].apply(convert_yoe)

# drop the rows with `yoe_total` or `yoe_at_company` being None
raw_df.dropna(subset=['yoe_total', 'yoe_at_company'], inplace=True)
shape = raw_df.shape

In [10]:
dtypes = raw_df.dtypes
dtypes

company                object
company_size            int32
job_title              object
level                  object
domain                 object
yoe_total             float64
yoe_at_company        float64
base                  float64
stock                 float64
bonus                 float64
total_compensation    float64
location               object
dtype: object

#### Missing values

In [11]:
# Check for null values
print('Number of null values in each column:')
raw_df.isnull().sum()

Number of null values in each column:


company                 0
company_size            0
job_title               0
level                   0
domain                  0
yoe_total               0
yoe_at_company          0
base                    0
stock                 330
bonus                 563
total_compensation      0
location                0
dtype: int64

In [12]:
def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

print('Missing values ratio:')
raw_df.apply(missing_ratio)

Missing values ratio:


company                0.0
company_size           0.0
job_title              0.0
level                  0.0
domain                 0.0
yoe_total              0.0
yoe_at_company         0.0
base                   0.0
stock                 26.2
bonus                 44.7
total_compensation     0.0
location               0.0
dtype: float64

The data is collected from levels.fyi, which is a crowdsourced website. The data is not verified by the companies, so it might be inaccurate. However, it is still a good source of information.

- There are jobs that are not related to software engineering, so we need to filter them out.
- There are jobs that does not have the salary information, so we need to filter them out.
- There are jobs that are the sum of base salary and bonus and stock is greater than the total compensation, so we need to filter them out.

#### For columns with numeric data type, how is the distribution of the data?

In [13]:
num_col_info_df = raw_df.select_dtypes(exclude='object')

def missing_ratio(s):
    return (s.isna().mean() * 100).round(1)

def median(df):
    return (df.quantile(0.5)).round(1)

def lower_quartile(df):
    return (df.quantile(0.25)).round(1)

def upper_quartile(df):
    return (df.quantile(0.75)).round(1)

num_col_info_df = num_col_info_df.agg([missing_ratio, "min", lower_quartile, median, upper_quartile, "max"])
num_col_info_df

,company_size,yoe_total,yoe_at_company,base,stock,bonus,total_compensation
missing_ratio,0.0,0.0,0.0,0.0,26.2,44.7,0.0
min,570.0,0.0,0.0,1100.0,1000.0,1000.0,6342.0
lower_quartile,19410.0,5.0,1.0,108975.0,24925.0,15000.0,133249.8
median,94520.0,9.0,2.0,157000.0,50000.0,23000.0,205000.0
upper_quartile,212570.0,13.0,4.0,200000.0,125000.0,38000.0,320500.0
max,865406.0,37.0,24.0,900000.0,750000.0,839900.0,2960000.0


From the above information, we have to deal with the missing values of `stock` and `bonus` columns
Although the missing ratio is high, we can accept it because they are optional to the company. Instead of dropping them, we can fill them with 0.

### Deal with duplicates

In [14]:
raw_df.drop_duplicates(inplace=True)
shape = raw_df.shape
print(f'Raw data shape after dropping duplicates: {shape}')

Raw data shape after dropping duplicates: (1258, 12)


### Deal with missing values

In [15]:
# Drop columns with more than 50% null values
raw_df.dropna(thresh=0.5*raw_df.shape[0], axis=1, inplace=True)
raw_df.shape

(1258, 12)

In [16]:
# Fill all the null values in domain column with 'Others'
raw_df['domain'].fillna('Others', inplace=True)
raw_df['domain'].isnull().sum()

0

In [17]:
# Fill all the NaN values in company_size column with the median value
raw_df['company_size'].fillna(raw_df['company_size'].median(), inplace=True)
raw_df['company_size'].isnull().sum()

0

In [18]:
# Fill all the NaN values in base, stock, bonus columns with 0
raw_df['base'].fillna(0, inplace=True)
raw_df['stock'].fillna(0, inplace=True)
raw_df['bonus'].fillna(0, inplace=True)
raw_df[['base', 'stock', 'bonus']].isnull().sum()

base     0
stock    0
bonus    0
dtype: int64

In [19]:
# Fill all the NaN values in total_compensation column with the sum of base, stock, bonus columns
raw_df['total_compensation'].fillna(raw_df['base'] + raw_df['stock'] + raw_df['bonus'], inplace=True)
raw_df['total_compensation'].isnull().sum()

0

In [20]:
# Re-check for null values
print('Number of null values in each column:')
raw_df.isnull().sum()

Number of null values in each column:


company               0
company_size          0
job_title             0
level                 0
domain                0
yoe_total             0
yoe_at_company        0
base                  0
stock                 0
bonus                 0
total_compensation    0
location              0
dtype: int64

In [21]:
raw_df.shape

(1258, 12)

In [22]:
# convert all the `float64` columns to `int64`
raw_df['base'] = raw_df['base'].astype(int)
raw_df['stock'] = raw_df['stock'].astype(int)
raw_df['bonus'] = raw_df['bonus'].astype(int)
raw_df['total_compensation'] = raw_df['total_compensation'].astype(int)
raw_df['yoe_total'] = raw_df['yoe_total'].astype(int)
raw_df['yoe_at_company'] = raw_df['yoe_at_company'].astype(int)

raw_df.dtypes

company               object
company_size           int32
job_title             object
level                 object
domain                object
yoe_total              int32
yoe_at_company         int32
base                   int32
stock                  int32
bonus                  int32
total_compensation     int32
location              object
dtype: object

### Is the collected data reasonable?

In [23]:
# total_compensation must be greater than or equal to base, stock, bonus
rows_with_wrong_total_compensation = raw_df[raw_df['total_compensation'] < raw_df['base'] + raw_df['stock'] + raw_df['bonus']]
print(f'Number of rows with wrong total_compensation: {rows_with_wrong_total_compensation.shape[0]}')
print(f'Ratio: {rows_with_wrong_total_compensation.shape[0] / raw_df.shape[0] * 100:.2f}%')
rows_with_wrong_total_compensation.head()

Number of rows with wrong total_compensation: 214
Ratio: 17.01%


,company,company_size,job_title,level,domain,yoe_total,yoe_at_company,base,stock,bonus,total_compensation,location
5,Logitech,7250,Software Engineer,I1,ML / AI,2,0,123100,0,0,123097,"New York, NY"
8,Logitech,7250,Software Engineer,I2,Web Development (Front-End),4,2,51500,0,4800,56250,"Cork, CK, Ireland"
9,Microsoft,182268,Software Engineer,60,Full Stack,2,2,24000,7200,2400,33576,"Noida, UP, India"
10,Microsoft,182268,Software Engineer,60,API Development (Back-End),4,2,25200,4000,3600,32786,"Hyderabad, TS, India"
11,Microsoft,182268,Software Engineer,Principal SDE,Full Stack,13,11,73400,38300,11700,123364,"Bengaluru, KA, India"


In [24]:
# Drop rows with wrong total_compensation
raw_df.drop(rows_with_wrong_total_compensation.index, inplace=True)
raw_df.shape

(1044, 12)

### Save the preprocessed data

In [25]:
raw_df.to_csv('./data/cleaned_data.csv', index=False)